In [25]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [2]:
model = AutoModelForSequenceClassification.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/')

Some weights of the model checkpoint at /shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/ were not used when initializing RobertaForSequenceClassification: ['roberta.embeddings.word_embeddings.0.weight', 'roberta.embeddings.word_embeddings.1.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier

In [5]:
tokenizer = AutoTokenizer.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/')

In [6]:
text = "Hell, let's print our own damn money! How expensive can a good printing press be?"

In [8]:
batch = tokenizer(text,return_tensors="pt",padding=True,truncation=True)

In [43]:
output = model(**batch, output_attentions=True, output_hidden_states=True)

In [152]:
model.roberta.encoder.layer[0].attention.self.value

Linear(in_features=768, out_features=768, bias=True)

In [123]:
batch.input_ids

tensor([[28254,    11,  1309,   338,  3601,   674,   898, 12270,  1637,     0,
          1374,  5789,   460,   257,   922, 13570,  1803,   307,    30]])

In [124]:
tokenizer.decode(12270)

' damn'

In [125]:
label_id=1
# to retain gradients
attention_weights = output['attentions']
weights_or = attention_weights[-2]
weights_or.retain_grad()

## output['logits'] : (batch_size, num_labels)
softmax=torch.nn.Softmax(dim=-1)
probs = softmax(output['logits'])[:, label_id]
# print(f"probs.shape:{probs.shape}")

probs.sum().backward(retain_graph=True)

In [126]:
# https://github.com/GChrysostomou/ood_faith/blob/145469f46670d7d24f79a9b2277e6c04914e00ba/src/models/deterministic/bert.py#L18
weights = weights_or[:, :, 0, :].mean(1)	

In [127]:
attentions = weights

In [128]:
weights_or.grad.shape

torch.Size([1, 12, 19, 19])

In [65]:
torch.norm(weights_or.grad[:,:,0,:],dim=1)

tensor([[0.0182, 0.0293, 0.0258, 0.0252, 0.0260, 0.0244, 0.0253, 0.0256, 0.0253,
         0.0246, 0.0269, 0.0269, 0.0284, 0.0282, 0.0302, 0.0280, 0.0288, 0.0234,
         0.0244]])

In [91]:
weights_or.grad[:,:,:,:].max(1)[0].shape

torch.Size([1, 19, 19])

In [78]:
torch.norm(weights_or.grad[:,:,:,:].mean(1),dim=2).shape

torch.Size([1, 19])

In [136]:
# attention_gradients = torch.norm(weights_or.grad[:,:,0,:],dim=0)
# attention_gradients = torch.norm(weights_or.grad[:,:,:,:].mean(1),dim=2)
# attention_gradients = torch.abs(weights_or.grad[:, :, 0, :].mean(1))
attention_gradients = weights_or.grad[:, :, 0, :].mean(1)
attention_gradients =  (attentions * attention_gradients)

In [129]:
# attention_gradients = torch.norm(weights_or.grad[:,:,0,:],dim=0)
# attention_gradients = torch.norm(weights_or.grad[:,:,:,:].mean(1),dim=2)
attention_gradients = torch.abs(weights_or.grad[:, :, 0, :].mean(1))
# attention_gradients = weights_or.grad[:, :, 0, :].mean(1)
attention_gradients =  (attentions * attention_gradients)

In [133]:
attention_gradients = torch.abs(weights_or.grad[:, :, 0, :].mean(1))
# attention_gradients = weights_or.grad[:, :, 0, :].mean(1)
attention_gradients =  (attentions / attention_gradients)

In [103]:
attentions

tensor([[0.1082, 0.0578, 0.0723, 0.0522, 0.0704, 0.0639, 0.0615, 0.0488, 0.0514,
         0.0578, 0.0368, 0.0329, 0.0285, 0.0321, 0.0321, 0.0438, 0.0370, 0.0572,
         0.0553]], grad_fn=<MeanBackward1>)

In [104]:
attention_gradients

tensor([[-0.0004, -0.0003, -0.0003, -0.0002, -0.0004, -0.0003, -0.0003, -0.0002,
         -0.0003, -0.0003, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002, -0.0002,
         -0.0002, -0.0002, -0.0003]], grad_fn=<MulBackward0>)

In [130]:
located_indices = torch.topk(attention_gradients,k=5)

In [131]:
located_indices

torch.return_types.topk(
values=tensor([[1.8283e-04, 1.0434e-04, 1.0371e-04, 9.3817e-05, 9.1121e-05]],
       grad_fn=<TopkBackward0>),
indices=tensor([[1, 4, 3, 5, 2]]))

In [132]:
tokenizer.decode(batch['input_ids'][0][located_indices[1].squeeze()])

", print's our let"

In [114]:
attentions

tensor([[0.1082, 0.0578, 0.0723, 0.0522, 0.0704, 0.0639, 0.0615, 0.0488, 0.0514,
         0.0578, 0.0368, 0.0329, 0.0285, 0.0321, 0.0321, 0.0438, 0.0370, 0.0572,
         0.0553]], grad_fn=<MeanBackward1>)

In [141]:
located_indices = torch.topk(attentions,k=5)

In [142]:
tokenizer.decode(batch['input_ids'][0][located_indices[1].squeeze()])

", our's print own"

In [143]:
located_indices = torch.topk(torch.abs(weights_or.grad[:, :, 0, :].mean(1)),k=5)

In [144]:
located_indices

torch.return_types.topk(
values=tensor([[0.0021, 0.0019, 0.0019, 0.0019, 0.0017]]),
indices=tensor([[14, 13, 12, 16, 15]]))

In [145]:
tokenizer.decode(batch['input_ids'][0][located_indices[1].squeeze()])

' good a can press printing'

In [137]:
located_indices = torch.topk(weights_or.grad[:, :, 0, :].mean(1),k=5)

In [138]:
weights_or.grad[:, :, 0, :].mean(1)

tensor([[-0.0016, -0.0016, -0.0013, -0.0013, -0.0013, -0.0010, -0.0011, -0.0013,
         -0.0014, -0.0012, -0.0016, -0.0016, -0.0019, -0.0019, -0.0021, -0.0017,
         -0.0019, -0.0010, -0.0012]])

In [139]:
located_indices

torch.return_types.topk(
values=tensor([[-0.0010, -0.0010, -0.0011, -0.0012, -0.0012]]),
indices=tensor([[17,  5,  6, 18,  9]]))

In [140]:
tokenizer.decode(batch['input_ids'][0][located_indices[1].squeeze()])

' be our own?!'

In [134]:
located_indices = torch.topk(attention_gradients,k=5)

In [135]:
tokenizer.decode(batch['input_ids'][0][located_indices[1].squeeze()])

" our, be's own"

In [ ]:
## value vector를 return 받는 코드
## https://github.com/gorokoba560/norm-analysis-of-transformer/blob/master/emnlp2020/transformers/src/transformers/modeling_bert.py
## explainable-qe-shared-task/model/sparse_xlmr.py 의 AttentionNormOutput